In [1]:
import os
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from scipy.stats import entropy, kurtosis
import xgboost as xgb
import seaborn as sns
from xgboost import plot_importance
import warnings
import matplotlib.pyplot as plt
import pandas as pd
from math import *
import numpy as np
import tensorflow as tf
import time

pd.set_option('display.max_columns', None)
from IPython.display import display

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
pathf = os.path.join("..", "data", "particles")
model_path = os.path.join(pathf, "model")
log_path = os.path.join(pathf, "model")
trainpd = pd.read_csv(os.path.join(pathf, "train.csv"))
print(trainpd.head(1))
trainshape = trainpd.shape
print(trainshape)
eventpd = pd.read_csv(os.path.join(pathf, "event.csv"))
print(eventpd.head(1))
print(eventpd.shape)
testpd = pd.read_csv(os.path.join(pathf, "test.csv"))
testshape = testpd.shape
print(testpd.head(1))
print(testpd.shape)

data = pd.concat([trainpd, testpd], ignore_index=True)
data = pd.merge(data, eventpd, on='event_id', how='left')

       x      y  z        t   terror        q  flag  event_id  hit_id
0 -142.5 -147.5  0  767.879  2.02966  1.05052     0         7       1
(9473201, 9)
   event_id  nhit  nhitreal  energymc  thetamc    phimc   xcmc    ycmc
0         7   426        70   48348.9  63.1686  11.0982 -40.83  114.03
(13315, 8)
       x      y  z        t  terror        q  event_id  hit_id
0 -142.5 -127.5  0  848.061  1.9984  1.15067         9       1
(4086511, 8)


In [3]:
#组合特征 
data['fx'] = data['x'] - data['xcmc']
data['fy'] = data['y'] - data['ycmc']
data['fdis'] = np.sqrt(data['fx'] ** 2 + data['fy'] ** 2)
# data['fscala'] = np.sin(data['thetamc']) * data['t']
data['fsinth'] = np.sin(data['thetamc'] * np.pi / 180.)
data['fcosth'] = np.cos(data['thetamc'] * np.pi / 180.)
data['fphi'] = np.arctan2(data['fy'], data['fx'])
data['fsinphi'] = np.sin(data['fphi'] * np.pi / 180.)
data['fcosphi'] = np.cos(data['fphi'] * np.pi / 180.)
data['fttrue'] = data['t'] / data['terror']
data['nhitratio'] = data['nhit'] / data['nhitreal']

# del data['fx']
# del data['fy']
del data['x']
del data['y']
del data['z']

In [4]:
info_new = pd.DataFrame()
info_new["event_id"] = data.groupby(["event_id"])["event_id"].mean()
info_new["fdis_stdmean"] = data.groupby(["event_id"])["fdis"].std() / data.groupby(["event_id"])["fdis"].mean()
info_new.reset_index(drop=True, inplace=True)
data = pd.merge(data, info_new, on='event_id', how='left')

In [5]:
trainpd = data[:trainshape[0]].reset_index()
testpd = data[trainshape[0]:].reset_index()
del data

In [6]:
print(trainpd.columns)
feature = [x for x in trainpd.columns if x not in ['flag', 'index', 'hit_id', 'event_id']]
labels = trainpd['flag']
del trainpd['flag']
del testpd['flag']

Index(['index', 'event_id', 'flag', 'hit_id', 'q', 't', 'terror', 'nhit',
       'nhitreal', 'energymc', 'thetamc', 'phimc', 'xcmc', 'ycmc', 'fx', 'fy',
       'fdis', 'fsinth', 'fcosth', 'fphi', 'fsinphi', 'fcosphi', 'fttrue',
       'nhitratio', 'fdis_stdmean'],
      dtype='object')


In [19]:
class AbstractModeltensor(object):
    def __init__(self, config=None):
        self.config = config

    # You need to override this method.
    def buildModel(self):
        raise NotImplementedError("You need to implement your own model.")


class NeurousNet(AbstractModeltensor):
    def __init__(self, xlenth, config=None):
        super(NeurousNet, self).__init__(config)
        self.graph = tf.Graph()  # 为每个类(实例)单独创建一个graph
        self.modeldic = {
            "cnn_dense_less": self._cnn_dense_less_model,
            "nomul_model": self._nomul_model,
        }
        self.ydim = 1
        self.keep_prob_ph = config["dropout"]
        self.input_dim = xlenth
        self.out_dim = 1
        with self.graph.as_default():
            with tf.name_scope('Inputs'):
                self.input_p = tf.placeholder(tf.float32, [None, self.input_dim])
                self.learn_rate_p = tf.placeholder(dtype=tf.float32, shape=[], name="lr")
                self.lr_decay = tf.placeholder(dtype=tf.float32, shape=[])
            with tf.name_scope('Outputs'):
                self.target_y = tf.placeholder(dtype=tf.float32, shape=[None, self.out_dim])

    def buildModel(self):
        tf.reset_default_graph()
        with self.graph.as_default():
            # 不同选择加载
            self.modeldic[self.config["modelname"]]()
            # 打印打包
            self.merged = tf.summary.merge_all()
            # 损失目标
            tvars = tf.trainable_variables()  # 返回需要训练的variable
            grads, _ = tf.clip_by_global_norm(tf.gradients(self.train_list, tvars), 2)
            grads_and_vars = tuple(zip(grads, tvars))
            self.train_op = tf.train.AdamOptimizer(self.learn_rate_p).apply_gradients(grads_and_vars)
            #             self.train_op = []
            #             for i2 in self.train_list:
            #                 self.train_op.append(tf.train.AdamOptimizer(self.learn_rate_p).minimize(i2))
            # 同一保存加载
            self.saver = tf.train.Saver(tf.global_variables())
            # [print(n.name) for n in tf.get_default_graph().as_graph_def().node]
            # return self.saver

    def _cnn_dense_less_model(self):
        with self.graph.as_default():
            # 部分1，预测值
            dense1 = tf.layers.dense(inputs=self.input_p, units=self.input_dim, activation=tf.nn.softmax,
                                     name="layer_dense1")
            tf.summary.histogram('dense1', dense1)  # 记录标量的变化
            mult_layer1 = tf.nn.softmax(dense1 * self.input_p, name='mult_layer1')
            mult_layer2 = tf.nn.softmax(mult_layer1 * self.input_p, name='mult_layer2')
            concat1 = tf.concat([self.input_p, dense1, mult_layer1, mult_layer2], 1, name='concat1')
            tf.summary.histogram('concat1', concat1)  # 记录标量的变化
            denseo1 = tf.nn.dropout(concat1, keep_prob=self.keep_prob_ph)
            denseo2 = tf.layers.dense(inputs=denseo1, units=self.input_dim, activation=tf.nn.elu, name="layer_dense2")
            denseo3 = tf.layers.dense(inputs=denseo2, units=self.input_dim // 4, activation=tf.nn.elu,
                                      name="layer_dense3")
            y_res_t = tf.layers.dense(inputs=denseo3, units=self.out_dim, activation=None)
            y_res_v = tf.nn.sigmoid(y_res_t, name="y_res_v")
            tf.summary.histogram('y_res_v', y_res_v)  # 记录标量的变化
            # 损失返回值
            y_los = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_res_t, labels=self.target_y, name="y_los")
            y_loss_t = tf.reduce_mean(y_los, name="y_loss_t")
            y_loss_v = tf.add(y_loss_t, 0, name="y_loss_v")

            one = tf.ones_like(y_res_t)
            zero = tf.zeros_like(y_res_t)
            label_bool = tf.where(y_res_t < 0.5, x=zero, y=one)
            self.auc_value, self.auc_op = tf.metrics.auc(self.target_y, label_bool, num_thresholds=4000)
            # 猜错的获取 实际盈利值的负数
            self.train_list = [y_loss_t]
            self.valid_list = [y_loss_v]
            self.pred_list = [y_res_v]
            # 打印信息
            tf.summary.scalar('y_loss_t', y_loss_t)  # 记录标量的变化
            tf.summary.scalar('y_loss_v', y_loss_v)  # 记录标量的变化
            tf.summary.histogram('mult_layer1', mult_layer1)  # 记录标量的变化
            tf.summary.histogram('mult_layer2', mult_layer2)  # 记录标量的变化

            tf.summary.scalar('lr', self.learn_rate_p)  # 记录标量的变化
            return None

    def _nomul_model(self):
        with self.graph.as_default():
            # 部分1，预测值
            dense1 = tf.layers.dense(inputs=self.input_p, units=self.input_dim, activation=tf.nn.softmax,
                                     name="layer_dense1")
            tf.summary.histogram('dense1', dense1)  # 记录标量的变化
            dense2 = tf.layers.dense(inputs=dense1, units=self.input_dim, activation=tf.nn.elu, name="layer_dense2")
            dense3 = tf.layers.dense(inputs=dense2, units=self.input_dim, activation=tf.nn.elu, name="layer_dense3")
            dense4 = tf.layers.dense(inputs=dense3, units=self.input_dim, activation=tf.nn.elu, name="layer_dense4")
            dense5 = tf.layers.dense(inputs=dense4, units=self.input_dim, activation=tf.nn.elu, name="layer_dense5")
            dense6 = tf.layers.dense(inputs=dense5, units=self.input_dim, activation=tf.nn.elu, name="layer_dense6")
            dense7 = tf.layers.dense(inputs=dense6, units=self.input_dim, activation=tf.nn.elu, name="layer_dense7")
            dense8 = tf.layers.dense(inputs=dense7, units=self.input_dim, activation=tf.nn.elu, name="layer_dense8")
            concat1 = tf.concat([self.input_p, dense1, dense2, dense3, dense4, dense5, dense6, dense7, dense8], 1,
                                name='concat1')
            tf.summary.histogram('concat1', concat1)  # 记录标量的变化
            denseo1 = tf.nn.dropout(concat1, keep_prob=self.keep_prob_ph)
            denseo2 = tf.layers.dense(inputs=denseo1, units=self.input_dim * 4, activation=tf.nn.elu,
                                      name="layer_denseo2")
            denseo3 = tf.layers.dense(inputs=denseo2, units=self.input_dim, activation=tf.nn.elu,
                                      name="layer_denseo3")
            denseo4 = tf.layers.dense(inputs=denseo3, units=self.input_dim // 4, activation=tf.nn.elu,
                                      name="layer_denseo4")
            y_res_t = tf.layers.dense(inputs=denseo4, units=self.out_dim, activation=None)
            y_res_v = tf.nn.sigmoid(y_res_t, name="y_res_v")
            tf.summary.histogram('y_res_v', y_res_v)  # 记录标量的变化
            # 损失返回值
            y_los = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_res_t, labels=self.target_y, name="y_los")
            y_loss_t = tf.reduce_mean(y_los, name="y_loss_t")
            y_loss_v = tf.add(y_loss_t, 0, name="y_loss_v")

            one = tf.ones_like(y_res_t)
            zero = tf.zeros_like(y_res_t)
            label_bool = tf.where(y_res_t < 0.5, x=zero, y=one)
            self.auc_value, self.auc_op = tf.metrics.auc(self.target_y, label_bool, num_thresholds=4000)
            # 猜错的获取 实际盈利值的负数
            self.train_list = [y_loss_t]
            self.valid_list = [y_loss_v]
            self.pred_list = [y_res_v]
            # 打印信息
            tf.summary.scalar('y_loss_t', y_loss_t)  # 记录标量的变化
            tf.summary.scalar('y_loss_v', y_loss_v)  # 记录标量的变化

            tf.summary.scalar('lr', self.learn_rate_p)  # 记录标量的变化
            return None

    def batch_train(self, trainpd, labels, batch_size=8, num_epochs=1, retrain=True):
        # 设置
        sess = tf.Session(graph=self.graph)
        with sess.as_default():
            with self.graph.as_default():
                if self.config["retrain"] == 1:
                    model_dir = os.path.join(model_path, "modelevery_%s" % self.config["tailname"])
                    latest_ckpt = tf.train.latest_checkpoint(model_dir)
                    if os.path.isfile("{}.index".format(latest_ckpt)):
                        self.saver.restore(sess, latest_ckpt)
                        sess.run(tf.local_variables_initializer())
                        print("retraining {}".format(latest_ckpt))
                    else:
                        sess.run(tf.global_variables_initializer())
                        sess.run(tf.local_variables_initializer())
                        print("no old model, training new----")
                writer = tf.summary.FileWriter(os.path.join(log_path, "logsevery_%s" % self.config["tailname"]),
                                               sess.graph)
                global_n = 0
                stop_n = 0
                startt = time.time()
                pre_t_base_loss = pre_t_much_loss = pre_v_much_loss = pre_v_base_loss = 100000

                n_splits = 5
                kf = KFold(n_splits=n_splits, shuffle=True, random_state=4389)
                for epoch in range(num_epochs):
                    self.config["learn_rate"] *= 0.3
                    trainevenidlist = list(set(trainpd['event_id']))
                    for train_index, valid_index in kf.split(trainevenidlist):
                        starte = time.time()
                        print("iter_trainnum", len(train_index))
                        np.random.shuffle(train_index)
                        np.random.shuffle(valid_index)
                        for batch_num, eventindex in enumerate(train_index):
                            # 获取数据
                            thisindex = trainpd[trainpd['event_id'] == trainevenidlist[eventindex]].index
                            r_inputs_t = np.array(trainpd.iloc[thisindex][feature])
                            r_output_t = np.expand_dims(np.array(labels[thisindex]), -1)
                            feed_dict_t = {
                                self.input_p: r_inputs_t,
                                self.target_y: r_output_t,
                                self.learn_rate_p: self.config["learn_rate"],
                                self.lr_decay: 1,
                            }
                            # 更新学习率
                            sess.run(self.train_op, feed_dict_t)
                            global_n += 1
                            losslist_t = sess.run(self.train_list, feed_dict_t)
                            sess.run(self.auc_op, feed_dict=feed_dict_t)
                            accu = sess.run(self.auc_value)
                            result = sess.run(self.merged, feed_dict_t)
                            if batch_num % 200 == 0:
                                writer.add_summary(result, global_n)
                                self.saver.save(sess,
                                                os.path.join(model_path, 'modelevery_%s' % self.config["tailname"],
                                                             self.config["modelfile"]), global_step=global_n)
                                print("epocht {}, batch_num {}, step {}, time: {} s, accu: {}, loss_yt: {}".format(
                                    epoch, batch_num, global_n, time.time() - starte, accu, *losslist_t))
                        # valid part
                        print("iter_validnum", len(valid_index))
                        losslist_va = 0
                        accu_va = 0
                        for batch_num, eventindex in enumerate(valid_index):
                            # 获取数据
                            thisindex = trainpd[trainpd['event_id'] == trainevenidlist[eventindex]].index
                            r_inputs_v = np.array(trainpd.iloc[thisindex][feature])
                            r_output_v = np.expand_dims(np.array(labels[thisindex]), -1)
                            feed_dict_v = {
                                self.input_p: r_inputs_v,
                                self.target_y: r_output_v,
                                self.learn_rate_p: self.config["learn_rate"],
                                self.lr_decay: 1,
                            }
                            losslist_v = sess.run(self.valid_list, feed_dict_v)
                            sess.run(self.auc_op, feed_dict=feed_dict_v)
                            accu = sess.run(self.auc_value)
                            losslist_va += losslist_v[0]
                            accu_va += accu
                        losslist_va /= len(valid_index)
                        accu_va /= len(valid_index)
                        result = sess.run(self.merged, feed_dict_v)
                        writer.add_summary(result, global_n)
                        if losslist_t[0] < pre_t_base_loss and losslist_va < pre_v_base_loss:
                            stop_n += 1
                            if stop_n > self.config["early_stop"]:
                                break
                            else:
                                self.saver.save(sess,
                                                os.path.join(model_path, 'modelevery_%s' % self.config["tailname"],
                                                             self.config["modelfile"]), global_step=global_n)
                        else:
                            stop_n = 0
                            self.saver.save(sess, os.path.join(model_path, 'modelevery_%s' % self.config["tailname"],
                                                               self.config["modelfile"]), global_step=global_n)
                        print("epochv {}, step {}, stop_n {}, time: {} s, accu_va: {}, loss_yv: {}".format(
                            epoch, global_n, stop_n, time.time() - starte, accu_va, losslist_va))
                        pre_t_base_loss = losslist_t[0]
                        pre_v_base_loss = losslist_va
                writer.close()
                print("total time: %s s" % (time.time() - startt))
        # 结束
        print("train finished!")
        return None

    def predict(self, inputs):
        model_dir = os.path.join(model_path, "modelevery_%s" % self.config["tailname"])
        print("loading model...")
        latest_ckpt = tf.train.latest_checkpoint(model_dir)

        sess = tf.Session(graph=self.graph)
        with sess.as_default():
            with self.graph.as_default():
                if os.path.isfile("{}.index".format(latest_ckpt)):
                    self.saver.restore(sess, latest_ckpt)
                else:
                    raise Exception("没有找到模型:{}".format(latest_ckpt))
                nplist = []
                oneiter = 2000
                redi = inputs.shape[0] % oneiter
                lenth = inputs.shape[0] // oneiter
                if 0 != redi:
                    lenth += 1
                counter = 0
                for num in range(lenth):
                    # 获取数据
                    startindex = num * oneiter
                    if num == lenth - 1 and redi != 0:
                        endindex = num * oneiter + redi
                    else:
                        endindex = (num + 1) * oneiter
                    tmppd = inputs.iloc[startindex:endindex][feature]
                    r_inputs_v = np.array(tmppd)
                    feed_dict = {
                        self.input_p: r_inputs_v,
                    }
                    teslis = sess.run(self.pred_list, feed_dict)
                    nplist.append(teslis)
                feed_dict = {
                    self.input_p: inputs,
                }
                teslist = np.concatenate(nplist, axis=1)
                return teslist


trainconfig = {
    "dropout": 0.5,
    "early_stop": 100,
#     "tailname": "nomul_modeltail",
#     "modelname": "nomul_model",
    "tailname": "mul_test",
    "modelname": "cnn_dense_less",
    "modelfile": "v2",
    "learn_rate": 0.01,
    "retrain": 1
}
modelcrnn = NeurousNet(len(feature), config=trainconfig)
modelcrnn.buildModel()

In [20]:
batch_size, num_epochs = 4096, 10000
print(trainpd.head())
globalstep = modelcrnn.batch_train(trainpd, labels, batch_size, num_epochs)

   index  event_id  hit_id          q         t   terror  nhit  nhitreal  \
0      0         7       1   1.050520  767.8790  2.02966   426        70   
1      1         7       2   0.999853  -70.5552  2.02966   426        70   
2      2         7       3   2.052540 -837.8410  1.85146   426        70   
3      3         7       4  19.513100 -973.1950  1.39994   426        70   
4      4         7       5   0.800334 -159.1400  2.02966   426        70   

   energymc  thetamc    phimc   xcmc    ycmc      fx      fy        fdis  \
0   48348.9  63.1686  11.0982 -40.83  114.03 -101.67 -261.53  280.597095   
1   48348.9  63.1686  11.0982 -40.83  114.03  -96.67 -266.53  283.519540   
2   48348.9  63.1686  11.0982 -40.83  114.03  -96.67 -246.53  264.805834   
3   48348.9  63.1686  11.0982 -40.83  114.03 -101.67 -231.53  252.869393   
4   48348.9  63.1686  11.0982 -40.83  114.03  -96.67 -231.53  250.900837   

     fsinth    fcosth      fphi   fsinphi   fcosphi      fttrue  nhitratio  \
0  0.892

epocht 0, batch_num 3200, step 10641, time: 122.49647855758667 s, accu: 0.8505020141601562, loss_yt: 0.24350520968437195
epocht 0, batch_num 3400, step 10841, time: 129.9495439529419 s, accu: 0.8505746126174927, loss_yt: 0.22349847853183746
epocht 0, batch_num 3600, step 11041, time: 137.39664816856384 s, accu: 0.8506602048873901, loss_yt: 0.33733245730400085
epocht 0, batch_num 3800, step 11241, time: 145.0830433368683 s, accu: 0.8505998253822327, loss_yt: 0.3111403286457062
epocht 0, batch_num 4000, step 11441, time: 152.54907989501953 s, accu: 0.8506993055343628, loss_yt: 0.49238941073417664
epocht 0, batch_num 4200, step 11641, time: 160.16672039031982 s, accu: 0.8507202863693237, loss_yt: 0.3479946553707123
epocht 0, batch_num 4400, step 11841, time: 167.88108444213867 s, accu: 0.8508158922195435, loss_yt: 0.277670294046402
epocht 0, batch_num 4600, step 12041, time: 175.54861402511597 s, accu: 0.8508191704750061, loss_yt: 0.3162747919559479
epocht 0, batch_num 4800, step 12241, t

epocht 0, batch_num 1200, step 23521, time: 46.11166477203369 s, accu: 0.8514974117279053, loss_yt: 0.5381332635879517
epocht 0, batch_num 1400, step 23721, time: 53.77417206764221 s, accu: 0.8515070676803589, loss_yt: 0.17125381529331207
epocht 0, batch_num 1600, step 23921, time: 61.404768228530884 s, accu: 0.851513683795929, loss_yt: 0.23237530887126923
epocht 0, batch_num 1800, step 24121, time: 68.93862318992615 s, accu: 0.8514977693557739, loss_yt: 0.40964871644973755
epocht 0, batch_num 2000, step 24321, time: 76.51536178588867 s, accu: 0.8515134453773499, loss_yt: 0.27907583117485046
epocht 0, batch_num 2200, step 24521, time: 84.237713098526 s, accu: 0.8515274524688721, loss_yt: 0.12859109044075012
epocht 0, batch_num 2400, step 24721, time: 91.77359390258789 s, accu: 0.8515454530715942, loss_yt: 0.2602071166038513
epocht 0, batch_num 2600, step 24921, time: 99.4580283164978 s, accu: 0.8515669107437134, loss_yt: 0.25924697518348694
epocht 0, batch_num 2800, step 25121, time: 1

epocht 0, batch_num 7200, step 36961, time: 272.55318427085876 s, accu: 0.852409839630127, loss_yt: 0.35412105917930603
epocht 0, batch_num 7400, step 37161, time: 280.1588795185089 s, accu: 0.852426290512085, loss_yt: 0.2678402364253998
iter_validnum 1860
epochv 0, step 37200, stop_n 0, time: 333.6348509788513 s, accu_va: 0.8522295066105422, loss_yv: 0.3281840598531148
iter_trainnum 7440
epocht 1, batch_num 0, step 37201, time: 0.31616711616516113 s, accu: 0.8519389629364014, loss_yt: 0.20100855827331543
epocht 1, batch_num 200, step 37401, time: 7.79217529296875 s, accu: 0.8519799709320068, loss_yt: 0.15820613503456116
epocht 1, batch_num 400, step 37601, time: 15.519510746002197 s, accu: 0.8519885540008545, loss_yt: 0.20871752500534058
epocht 1, batch_num 600, step 37801, time: 23.076273202896118 s, accu: 0.8519834280014038, loss_yt: 0.3163989186286926
epocht 1, batch_num 800, step 38001, time: 30.60115623474121 s, accu: 0.8520246148109436, loss_yt: 0.399379700422287
epocht 1, batch

epocht 1, batch_num 5200, step 49841, time: 196.50557160377502 s, accu: 0.854060709476471, loss_yt: 0.6710466742515564
epocht 1, batch_num 5400, step 50041, time: 204.2508363723755 s, accu: 0.8540775179862976, loss_yt: 0.46258479356765747
epocht 1, batch_num 5600, step 50241, time: 211.71387362480164 s, accu: 0.8541157245635986, loss_yt: 0.2503279149532318
epocht 1, batch_num 5800, step 50441, time: 219.12605214118958 s, accu: 0.8541485071182251, loss_yt: 0.3085162937641144
epocht 1, batch_num 6000, step 50641, time: 226.6499330997467 s, accu: 0.8541585206985474, loss_yt: 0.38837283849716187
epocht 1, batch_num 6200, step 50841, time: 234.10503029823303 s, accu: 0.8542103171348572, loss_yt: 0.2951407730579376
epocht 1, batch_num 6400, step 51041, time: 241.82734870910645 s, accu: 0.8542119860649109, loss_yt: 0.3401823341846466
epocht 1, batch_num 6600, step 51241, time: 249.26446056365967 s, accu: 0.8542420864105225, loss_yt: 0.23706597089767456
epocht 1, batch_num 6800, step 51441, ti

...

epocht 17, batch_num 3600, step 643441, time: 138.51962733268738 s, accu: 0.8634845614433289, loss_yt: 0.3469860255718231
epocht 17, batch_num 3800, step 643641, time: 146.07141709327698 s, accu: 0.8634840250015259, loss_yt: 0.2702072262763977
epocht 17, batch_num 4000, step 643841, time: 153.7319188117981 s, accu: 0.8634823560714722, loss_yt: 0.3034464716911316
epocht 17, batch_num 4200, step 644041, time: 161.21395349502563 s, accu: 0.8634828925132751, loss_yt: 0.46499747037887573
epocht 17, batch_num 4400, step 644241, time: 168.95421314239502 s, accu: 0.8634838461875916, loss_yt: 0.11841616034507751
epocht 17, batch_num 4600, step 644441, time: 176.69152331352234 s, accu: 0.8634849786758423, loss_yt: 0.26865869760513306
epocht 17, batch_num 4800, step 644641, time: 184.51061391830444 s, accu: 0.8634840846061707, loss_yt: 0.16832289099693298
epocht 17, batch_num 5000, step 644841, time: 191.9667043685913 s, accu: 0.8634838461875916, loss_yt: 0.3561689555644989
epocht 17, batch_num 5

epocht 17, batch_num 1400, step 656121, time: 54.571101903915405 s, accu: 0.8634929656982422, loss_yt: 0.2338535487651825
epocht 17, batch_num 1600, step 656321, time: 62.50186729431152 s, accu: 0.8634929656982422, loss_yt: 0.35367369651794434
epocht 17, batch_num 1800, step 656521, time: 70.14842057228088 s, accu: 0.8634927272796631, loss_yt: 0.35580721497535706
epocht 17, batch_num 2000, step 656721, time: 77.70723724365234 s, accu: 0.8634927868843079, loss_yt: 0.2063572108745575
epocht 17, batch_num 2200, step 656921, time: 85.18326330184937 s, accu: 0.8634934425354004, loss_yt: 0.22555488348007202
epocht 17, batch_num 2400, step 657121, time: 92.98937106132507 s, accu: 0.8634952306747437, loss_yt: 0.25569239258766174
epocht 17, batch_num 2600, step 657321, time: 101.26820659637451 s, accu: 0.8634949922561646, loss_yt: 0.3682571351528168
epocht 17, batch_num 2800, step 657521, time: 108.72330379486084 s, accu: 0.8634958863258362, loss_yt: 0.3020014762878418
epocht 17, batch_num 3000

epocht 17, batch_num 7000, step 669161, time: 266.3278286457062 s, accu: 0.863513171672821, loss_yt: 0.26531273126602173
epocht 17, batch_num 7200, step 669361, time: 273.8716604709625 s, accu: 0.8635141849517822, loss_yt: 0.24102218449115753
epocht 17, batch_num 7400, step 669561, time: 281.3287148475647 s, accu: 0.8635151982307434, loss_yt: 0.25855615735054016
iter_validnum 1860
epochv 17, step 669600, stop_n 0, time: 334.8955078125 s, accu_va: 0.8635157040049953, loss_yv: 0.29839535982057613
iter_trainnum 7440
epocht 18, batch_num 0, step 669601, time: 0.5904543399810791 s, accu: 0.8635184168815613, loss_yt: 0.4658547043800354
epocht 18, batch_num 200, step 669801, time: 8.188105821609497 s, accu: 0.8635191917419434, loss_yt: 0.16841986775398254
epocht 18, batch_num 400, step 670001, time: 15.658163070678711 s, accu: 0.863518476486206, loss_yt: 0.24174617230892181
epocht 18, batch_num 600, step 670201, time: 23.201958179473877 s, accu: 0.8635186553001404, loss_yt: 0.3553318679332733

epocht 18, batch_num 4800, step 681841, time: 183.75861811637878 s, accu: 0.8635333180427551, loss_yt: 0.28763097524642944
epocht 18, batch_num 5000, step 682041, time: 191.452045917511 s, accu: 0.8635337948799133, loss_yt: 0.3786075711250305
epocht 18, batch_num 5200, step 682241, time: 199.05870723724365 s, accu: 0.8635343313217163, loss_yt: 0.2664293348789215
epocht 18, batch_num 5400, step 682441, time: 206.71127557754517 s, accu: 0.8635351657867432, loss_yt: 0.37143048644065857
epocht 18, batch_num 5600, step 682641, time: 214.24709224700928 s, accu: 0.8635345697402954, loss_yt: 0.319668173789978
epocht 18, batch_num 5800, step 682841, time: 222.00235390663147 s, accu: 0.8635349869728088, loss_yt: 0.2879188060760498
epocht 18, batch_num 6000, step 683041, time: 229.7187533378601 s, accu: 0.8635357022285461, loss_yt: 0.2536905109882355
epocht 18, batch_num 6200, step 683241, time: 237.3303678035736 s, accu: 0.8635374307632446, loss_yt: 0.5349134802818298
epocht 18, batch_num 6400, 

KeyboardInterrupt: 

In [15]:
y_pred = modelcrnn.predict(testpd[feature])
y_pred = np.squeeze(y_pred)

loading model...
INFO:tensorflow:Restoring parameters from ..\data\particles\model\modelevery_nomul_modeltail\v2-20881


In [16]:
#阈值大概在0.2-0.4之间 本题对召回率较敏感，可适当降低一下阈值
thre = 0.5
#生成提交文件
sub = pd.DataFrame()
sub['hit_id'] = testpd['hit_id']
sub['flag_pred'] = y_pred
sub['event_id'] = testpd['event_id']
sub['flag_pred'] = sub['flag_pred'].apply(lambda x: 1 if x >= thre else 0)
sub.to_csv(os.path.join(pathf, "subsample.csv").format(sub['flag_pred'].mean()), index=False)

KeyboardInterrupt: 